## Multiprocessing

In [1]:
import os

print('Process (%s) start...' % os.getpid())
# Only works on Unix/Linux/Mac:
pid = os.fork()
if pid == 0:
    print('I am child process (%s) and my parent is %s.' % (os.getpid(), os.getppid()))
else:
    print('I (%s) just created a child process (%s).' % (os.getpid(), pid))

Process (7841) start...
I (7841) just created a child process (7852).
I am child process (7852) and my parent is 7841.


In [3]:
# importing the multiprocessing module 
import multiprocessing 
import os 
  
def worker1(): 
    # printing process id 
    print("ID of process running worker1: {}".format(os.getpid())) 
  
def worker2(): 
    # printing process id 
    print("ID of process running worker2: {}".format(os.getpid())) 
  
if __name__ == "__main__": 
    # printing main program process id 
    print("ID of main process: {}".format(os.getpid())) 
  
    # creating processes 
    p1 = multiprocessing.Process(target=worker1) 
    p2 = multiprocessing.Process(target=worker2) 
  
    # starting processes 
    p1.start() 
    p2.start() 
  
    # process IDs 
    print("ID of process p1: {}".format(p1.pid)) 
    print("ID of process p2: {}".format(p2.pid)) 
    
        # check if processes are alive 
    print("Process p1 is alive: {}".format(p1.is_alive())) 
    print("Process p2 is alive: {}".format(p2.is_alive()))
  
    # wait until processes are finished 
    p1.join() 
    p2.join() 
  
    # both processes finished 
    print("Both processes finished execution!") 
  
    # check if processes are alive 
    print("Process p1 is alive: {}".format(p1.is_alive())) 
    print("Process p2 is alive: {}".format(p2.is_alive())) 

ID of main process: 7841
ID of process running worker1: 7909
ID of process running worker2: 7910
ID of process p1: 7909
ID of process p2: 7910
Process p1 is alive: True
Process p2 is alive: True
Both processes finished execution!
Process p1 is alive: False
Process p2 is alive: False


In multiprocessing, workers
- run independently
- have their own memory space

In [6]:
import multiprocessing

result = []

def square_list(mylist): 
    """ 
    function to square a given list 
    """
    global result 
    # append squares of mylist to global list result 
    for num in mylist: 
        result.append(num * num) 
    # print global list result 
    print("Result(in process p1): {}".format(result))

l=[1,2,3,4]
p1 = multiprocessing.Process(target=square_list, args=(l,))
p1.start()
p1.join()

# print global result list 
print("Result(in main program): {}".format(result))  # cannot directly access the data in subprocess

Result(in process p1): [1, 4, 9, 16]
Result(in main program): []


## Solution 1: share memory
- `Array`: a ctypes array allocated from **shared memory**
- `Value`: a ctypes object allocated from **shared memory**

In [18]:
import multiprocessing

def square_list(mylist, result, square_sum): 
    """ 
    function to square a given list 
    """
    # append squares of mylist to result array 
    for idx, num in enumerate(mylist): 
        result[idx] = num * num 
    
    # square sum value
    square_sum.value = sum(result)
    print("Result(in process p1): {}".format(result[:])) 
    # print square_sum Value 
    print("Sum of squares(in process p1): {}".format(square_sum.value))
    
l = [1,2,3,4]

result_array = multiprocessing.Array('i', 4) # create array of int with 4 integers
square_sum_value = multiprocessing.Value('i')

p1 = multiprocessing.Process(target=square_list, args=(l, result_array, square_sum_value))

p1.start()
p1.join()

# print result array 
print("Result(in main program): {}".format(result_array[:])) 
  
# print square_sum Value 
print("Sum of squares(in main program): {}".format(square_sum_value.value)) 

Result(in process p1): [1, 4, 9, 16]
Sum of squares(in process p1): 30
Result(in main program): [1, 4, 9, 16]
Sum of squares(in main program): 30


## Solution 2: server process
Use `multiprocessing.Manager`
- support arbitrary object types like lists, dictionaries, Queue, Value, Array, etc.
- slower than shared memory

In [20]:
import multiprocessing 
  
def print_records(records): 
    """ 
    function to print record(tuples) in records(list) 
    """
    for record in records: 
        print("Name: {0}\nScore: {1}\n".format(record[0], record[1]))

def insert_record(record, records): 
    """ 
    function to add a new record to records(list) 
    """
    records.append(record) 
    print("New record added!\n")

with multiprocessing.Manager() as manager:
    # creating a list in server process memory 
    records = manager.list([('a',10), ('b',9), ('k',8)])
    new_record = ("J", 7)
    p1 = multiprocessing.Process(target=insert_record, args=(new_record, records))
    p2 = multiprocessing.Process(target=print_records, args=(records,))
    
    p1.start()
    p1.join()
    
    p2.start()
    p2.join()

New record added!

Name: a
Score: 10

Name: b
Score: 9

Name: k
Score: 8

Name: J
Score: 7

